## Train nn using pytorch lightning

In [1]:
# essentials
import os
import pathlib

import pandas as pd
import numpy as np
from tqdm import tqdm

# visualisation
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn imports
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MaxAbsScaler, PowerTransformer, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, SequentialFeatureSelector, RFECV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.base import clone as clone_model
from sklearn.metrics import classification_report, confusion_matrix, log_loss


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# others
import xgboost as xgb 
import lightgbm as lgb
import catboost as cb

# pytorch lightning
import torch, torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import lightning as L

RANDOM_SEED = 64

ModuleNotFoundError: No module named 'torch'

In [ ]:
IN_KAGGLE = False
kaggle_folder = "/kaggle/input/playground-series-s3e26"
local_folder = "./data"
train_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "/train.csv", index_col="id")
test_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder  + "/test.csv", index_col="id")

target_column = "Status"

def feature_engineering(df):
    df['Drug'] = df['Drug'].map({"D-penicillamine": 1,"placebo": 0})
    df['Sex'] = df['Sex'].map({"F": 1,"M": 0})

    # change "Stage" to string
    df["Stage"] = df["Stage"].apply(lambda x: str(x))
    return df

run_feature_engineering = True

categorical_features = ["Drug", "Sex", "Ascites", "Hepatomegaly", "Spiders", "Edema", "Stage"]
numerical_features = ["N_Days", "Age", "Bilirubin", "Cholesterol", "Albumin", "Copper", "Alk_Phos", "SGOT", "Tryglicerides", "Platelets", "Prothrombin"]

if run_feature_engineering:
    train_df = feature_engineering(train_df)

if run_feature_engineering:
    categorical_features += ["no_diseases", "diseases"]
    numerical_features += ["date_of_diagnosis"]

X = train_df.drop(columns=target_column)
y = train_df[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=RANDOM_SEED, stratify=y, shuffle=True)

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

In [ ]:
numeric_transformer = Pipeline(
    [
        ("power_transformer", PowerTransformer()),
        ("scaler", MaxAbsScaler()),
    ]
)

categorical_transformer = Pipeline(
    [
        ("onehot", OneHotEncoder(handle_unknown="ignore", drop="if_binary")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

models = [
    ("default xgboost", xgb.XGBClassifier(
        objective="multi:softprob", random_state=RANDOM_SEED, n_jobs=-1 )),
    ("optimized_xgboost", xgb.XGBClassifier(
        objective="multi:softprob", random_state=RANDOM_SEED, n_jobs=-1,
        subsample=0.8,
        min_child_weight=7,
        max_depth=7,
        reg_lambda=0.9,
        gamma=0.9,
        eta=0.08,
        colsample_bytree=0.5,
        reg_alpha=0.5
    )),
    ("default catboost", cb.CatBoostClassifier(random_state=RANDOM_SEED, verbose=False)),
    ("default lightgbm", lgb.LGBMClassifier(random_state=RANDOM_SEED, n_jobs=4)),
    

]


clf = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", CalibratedClassifierCV(model, cv=skf)),
    ]
)

clf.fit(X_train, y_train)
y_pred_proba = clf.predict_proba(X_val)
y_pred = clf.predict(X_val)

log_loss_score = log_loss(y_val, y_pred_proba)